In [1]:
# !pip install librosa

In [1]:
import pandas as pd
import numpy as np
import librosa

# Read the Feather file
df = pd.read_feather('CRSP_daily_data_for_project(Technical_Analysis).feather')
# df = df[(df['PERMNO']==10000) |( df['PERMNO']==10001)|(df['PERMNO']==10002)| (df['PERMNO']==93434)|(df['PERMNO']==93435)|(df['PERMNO']==93436)]


In [2]:
df[df['PERMNO']==10000]

,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn
1,10000,1986-01-07,NaN,2.750,2.3750,2.56250,1000.0,3680.0,0.013800,0.011046,0.014954
2,10000,1986-01-08,NaN,2.625,2.3750,2.50000,12800.0,3680.0,-0.020750,-0.005135,-0.027268
3,10000,1986-01-09,NaN,2.625,2.3750,2.50000,1400.0,3680.0,-0.011315,-0.011659,-0.008944
4,10000,1986-01-10,NaN,2.625,2.3750,2.50000,8500.0,3680.0,0.000047,0.003632,-0.000728
5,10000,1986-01-13,NaN,2.750,2.5000,2.62500,5450.0,3680.0,0.002680,0.002369,0.003690
...,...,...,...,...,...,...,...,...,...,...,...
359,10000,1987-06-08,NaN,0.250,0.1875,0.21875,0.0,3893.0,0.008563,0.001508,0.011143
360,10000,1987-06-09,NaN,0.250,0.1875,0.21875,0.0,3893.0,0.001918,0.002217,0.001887
361,10000,1987-06-10,NaN,0.250,0.1875,0.21875,0.0,3893.0,0.001492,0.001049,0.000639
362,10000,1987-06-11,NaN,0.250,0.1875,0.21875,500.0,3893.0,0.003427,0.002650,0.004236


In [3]:
from datetime import datetime
df['date']=df['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [ ]:
df['date'].max()

Timestamp('2023-12-29 00:00:00')

In [4]:
# Function: Generate CQT data for each stock
def create_cqt_df(stock_df, n_bins=20):
    stock_returns = stock_df['vwretx'].values

    # Data preprocessing: Ensure all values are finite
    stock_returns = np.nan_to_num(stock_returns, nan=0.0, posinf=0.0, neginf=0.0)

    sr = 1
    fmin = 0.01  # Minimum frequency
    hop_length = 1
    cqt_result = librosa.cqt(stock_returns, n_bins=n_bins, sr=sr, hop_length=hop_length, fmin=fmin)
    cqt_result_db = librosa.amplitude_to_db(np.abs(cqt_result), ref=np.mean)

    cqt_df = pd.DataFrame(cqt_result_db.T, columns=[f'CQT_{i+1}' for i in range(cqt_result_db.shape[0])])
    cqt_df = cqt_df.loc[:len(stock_df)-1, :]  # Ensure the length matches the original data
    cqt = pd.concat([stock_df['date'], cqt_df], axis=1)
    return cqt

# Function: Create rolling windows and retain date indices
def create_rolling_windows_with_dates(data, cqt_window_size, date_to_index):
    windows = []
    date_indices = []
    for i in range(len(data) - cqt_window_size + 1):
        window = data.iloc[i:i + cqt_window_size, 1:].values  # Exclude the date column, keep only CQT data
        windows.append(window)
        start_date = data.iloc[i]['date']
        if start_date in date_to_index:
            date_indices.append(date_to_index[start_date])
    return np.array(windows), date_indices


In [48]:
def get_pre_processed_data(df, start_date, end_date, window_size=30):
    
    df_sub = df[(df['date']>=start_date) & (df['date']<=end_date)]

    # Get the list of all stock codes
    stock_codes = set(df_sub['PERMNO'].unique())
    # stock_codes = set()

    cqt_window_size = window_size
    n_bins = 20  # Number of CQT bins

    # Get all unique dates
    unique_dates = df_sub['date'].unique()

    # drop the PERMNO for which data is not available on all unique_dates
    for _,r in df_sub.groupby(['date']):
        stock_codes.intersection_update(set(r['PERMNO'].to_list()))

    stock_codes = list(stock_codes)
    
    df_sub = df_sub[df_sub['PERMNO'].isin(stock_codes)]
    
    # Create the 4D array, initialized with NaN
    num_dates = len(unique_dates)
    max_shape = (len(stock_codes), num_dates - cqt_window_size + 1, cqt_window_size, n_bins)
    final_4d_array = np.full(max_shape, np.nan)

    # Create the date-to-index mapping
    date_to_index = {date: idx for idx, date in enumerate(unique_dates)}

    # Create the stock code-to-index mapping
    stock_code_to_index = {stock_code: idx for idx, stock_code in enumerate(stock_codes)}

    # Iterate over each stock and fill the 4D array
    for stock_code in stock_codes:
        stock_idx = stock_code_to_index[stock_code]
        stock_data = df_sub[df_sub['PERMNO'] == stock_code].reset_index(drop=True)
        
        # Generate CQT data
        cqt_data = create_cqt_df(stock_data, n_bins)

        # Create rolling windows and retain date indices
        rolling_windows_3d, date_indices = create_rolling_windows_with_dates(cqt_data, cqt_window_size, date_to_index)
        
        # Fill the 4D array
        for window_idx, (window_data, date_idx) in enumerate(zip(rolling_windows_3d, date_indices)):
            if window_data.shape == (cqt_window_size, n_bins):
                final_4d_array[stock_idx, date_idx, :, :] = window_data

    df_sub = df_sub.sort_values(by=['PERMNO', 'date'])
    df_sub['Ret'] = df_sub.groupby('PERMNO')['Close'].apply(lambda x: x.shift(-30).pct_change()).reset_index(level=0, drop=True)


    return final_4d_array, df_sub, stock_codes

In [39]:
df['date'].min()

Timestamp('1969-01-02 00:00:00')

In [40]:
start_date = df['date'].min()
end_date = df['date'].max()

In [6]:
# start_date = datetime.strptime("2000-01-01", "%Y-%m-%d")
# end_date = datetime.strptime("2001-01-01", "%Y-%m-%d")

In [16]:
import numpy as np
# final_4d_array, df_sub, stock_codes = get_pre_processed_data(df, start_date, end_date)

In [17]:
# stock=0
# stock_data = final_4d_array[stock]
# ret = df_sub[df_sub.PERMNO == stock_codes[stock]]['Ret'].values[-stock_data.shape[0]:].reshape(-1, 1)

In [18]:
# model = Sequential()
# model.add(LSTM(50, input_shape=(final_4d_array.shape[1], 1)))
# model.add(Dropout(0.2))
# model.add(Dense(1))
# model.compile(loss='mean_squared_error', optimizer='adam')

In [19]:
# scaler = MinMaxScaler()
# stock_data = scaler.fit_transform(stock_data.reshape(stock_data.shape[0], stock_data.shape[1] * stock_data.shape[2]))

# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(stock_data, ret, test_size=0.2, shuffle=False)


In [20]:
# stock_data.shape

In [21]:
# X_test.shape

In [12]:
# df_sub

In [13]:
# df_sub[df_sub['PERMNO']==32791]

In [14]:
# stock_codes

In [15]:
# final_4d_array.shape

In [16]:
# !pip install keras tensorflow

In [17]:
# stock_codes

In [22]:
# Apply LSTM model to the CQT data
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [56]:
def apply_lstm_model(final_4d_array, df_sub, stock_codes, date):
    model = Sequential()
    model.add(LSTM(50, input_shape=(final_4d_array.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # make an array to store all X_test
    X_test_array = None
    y_test_array = None
    X_train_array = None
    y_train_array = None

    for stock in range(final_4d_array.shape[0]):
        stock_data = final_4d_array[stock]
        ret = df_sub[df_sub.PERMNO == stock_codes[stock]]['Ret'].values[-stock_data.shape[0]:].reshape(-1, 1)
        # Normalize the data
        scaler = MinMaxScaler()
        stock_data = scaler.fit_transform(stock_data.reshape(stock_data.shape[0], stock_data.shape[1] * stock_data.shape[2]))

        # Split the data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(stock_data, ret, test_size=0.2, shuffle=False)

        if X_test_array is None:
            X_test_array = np.zeros((final_4d_array.shape[0], X_test.shape[0], X_test.shape[1]))
        if X_train_array is None:
            X_train_array = np.zeros((final_4d_array.shape[0], X_train.shape[0], X_train.shape[1]))
        
        if y_test_array is None:
            y_test_array = np.zeros((final_4d_array.shape[0], y_test.shape[0], y_test.shape[1]))
        if y_train_array is None:
            y_train_array = np.zeros((final_4d_array.shape[0], y_train.shape[0], y_train.shape[1]))
            
        X_test_array[stock] = X_test
        X_train_array[stock] = X_train
        y_test_array[stock] = y_test
        y_train_array[stock] = y_train

    X_train_array = X_train_array.reshape(X_train_array.shape[0] * X_train_array.shape[1], X_train_array.shape[2])
    y_train_array = y_train_array.reshape(y_train_array.shape[0] * y_train_array.shape[1], y_train_array.shape[2])
    # Train the model
    train_loss = model.fit(X_train_array, y_train_array, epochs=10, batch_size=32, verbose=0)
    test_loss = model.evaluate(X_test_array, y_test_array, batch_size=32, verbose=0)
    
    print("Train loss:", train_loss)
    print("Test loss:", test_loss)

    for stock in range(final_4d_array.shape[0]):
        predictions = model.predict(X_test_array[stock])
        # save numpy array to disk
        np.save(f'predictions/{date.date().strftime("%Y%m%d")}_{stock_codes[stock]}_test_predictions.npy', predictions)

    return model

In [20]:
# model = apply_lstm_model(final_4d_array, df_sub, stock_codes)
# model.summary()

In [ ]:
# save keras model
# model.save('lstm_model.h5')

In [36]:
from dateutil.relativedelta import relativedelta

In [57]:
def train_model_annually(df, start_date, end_date, window_size=30):
    # models = []
    # iterate between start_date and end_date with 1 year step size
    while start_date < end_date:
        next_year = start_date + relativedelta(years=1)
        print(start_date, next_year)
        if next_year > end_date:
            next_year = end_date
        final_4d_array, df_sub, stock_codes = get_pre_processed_data(df, start_date, next_year, window_size)
        model = apply_lstm_model(final_4d_array, df_sub, stock_codes, start_date)
        model.save(f'models/lstm_model_{start_date.date().strftime("%Y%m%d")}.keras')
        start_date = next_year
        # models.append(model)
    # return models

In [54]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

In [58]:
train_model_annually(df, start_date, end_date, window_size=30)

1969-01-02 00:00:00 1970-01-02 00:00:00
